In [1]:
# Importar bibliotecas necesarias
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import os
import numpy as np


In [2]:
def preprocess_signal(signal):

    # Parámetros teóricos
    max_theoretical_value = 230 * np.sqrt(2)

    # Media de la señal
    mean_value = np.mean(signal)

    # Datos sin sesgo
    unbias_data = signal - mean_value
    unbias_data_2 = unbias_data ** 2
    unbias_data_3 = unbias_data_2 * unbias_data
    unbias_data_4 = unbias_data_3 * unbias_data

    # Cálculo de características
    variance = np.var(unbias_data)  # Varianza
    skewness = np.mean(unbias_data_3) / (variance ** 1.5)  # Asimetría
    kurtosis = np.mean(unbias_data_4) / (variance ** 2) - 3  # Curtosis
    thd = np.sqrt(np.sum(np.abs(np.fft.fft(signal)[2:4])) / np.abs(np.fft.fft(signal)[1]))  # Distorsión armónica total
    rms = np.sqrt(np.mean(signal ** 2))  # Valor RMS
    crest_factor = np.max(signal) / rms  # Factor de cresta

    # Devuelve todas las características en un vector
    return np.array([variance, skewness, kurtosis, thd, crest_factor])

def load_signal(data_path):

    # Asignar etiquetas explícitamente
    label_mapping = {
        "flicker_signals": 0,
        "harmonic_signals": 1,
        "interruption_signals": 2,
        "original_signals": 3,
        "sag_signals": 4,
        "swell_signals": 5,
        "transient_signals": 6,
    }

    # Inicialización de listas para características y etiquetas
    features = []
    labels = []

    # Iterar por cada tipo de perturbación
    for signal_type, label in label_mapping.items():
        signal_type_path = os.path.join(data_path, signal_type)

        if os.path.isdir(signal_type_path):
            for subset in ["train", "test", "val"]:
                subset_path = os.path.join(signal_type_path, subset)

                if os.path.exists(subset_path):
                    for filename in os.listdir(subset_path):
                        if filename.endswith(".npy"):
                            file_path = os.path.join(subset_path, filename)

                            # Cargar la señal
                            signal = np.load(file_path)

                            # Procesar la señal y extraer características
                            feature_vector = preprocess_signal(signal)

                            # Agregar las características y etiquetas a las listas
                            features.append(feature_vector)
                            labels.append(label)

    return np.array(features), np.array(labels)

# Ejemplo de uso
data_path = "data"  # Ajusta esta ruta según sea necesario
features, labels = load_signal(data_path)

print(f"Características extraídas: {features.shape}")
print(f"Etiquetas extraídas: {labels.shape}")


# Contar las etiquetas únicas en los datos originales 
unique_labels, counts = np.unique(labels, return_counts=True)
print("Etiquetas únicas:", unique_labels)
print("Distribución de señales por categoría:")
for label, count in zip(unique_labels, counts):
    print(f"Clase {label}: {count} señales")

# Verificar que la clase 3 está presente
if 3 not in unique_labels: 
    print("Error: La clase 3 no está presente en los datos originales.")

Características extraídas: (75600, 5)
Etiquetas extraídas: (75600,)
Etiquetas únicas: [0 1 2 3 4 5 6]
Distribución de señales por categoría:
Clase 0: 10800 señales
Clase 1: 10800 señales
Clase 2: 10800 señales
Clase 3: 10800 señales
Clase 4: 10800 señales
Clase 5: 10800 señales
Clase 6: 10800 señales


In [3]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir los rangos de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],       # Valores bajos a altos de C
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],  # Gamma ajustado
    'kernel': ['rbf']             # Mantener kernel RBF
}

# Crear el modelo SVM
svm = SVC()

# Configurar búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')

# Entrenar el modelo y realizar búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y rendimiento
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación (validación cruzada):", grid_search.best_score_)

# Usar el mejor modelo para predicciones
best_svm = grid_search.best_estimator_
predicciones = best_svm.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, predicciones)
print(f"Accuracy en el conjunto de prueba: {accuracy:.2f}")

# Generar el informe de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, predicciones))

# Matriz de confusión
print("\nMatriz de Confusión:")
conf_matrix = confusion_matrix(y_test, predicciones)
print(conf_matrix)

Mejores parámetros: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Mejor puntuación (validación cruzada): 0.9825727513227515
Accuracy en el conjunto de prueba: 0.98

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2154
           1       0.96      0.96      0.96      2134
           2       0.97      1.00      0.98      2193
           3       1.00      1.00      1.00      2158
           4       0.99      0.96      0.97      2146
           5       0.98      0.98      0.98      2187
           6       1.00      0.99      0.99      2148

    accuracy                           0.98     15120
   macro avg       0.98      0.98      0.98     15120
weighted avg       0.98      0.98      0.98     15120


Matriz de Confusión:
[[2154    0    0    0    0    0    0]
 [  23 2041    2    1   21   46    0]
 [   0    3 2188    1    1    0    0]
 [   0    2    0 2156    0    0    0]
 [   0   18   69    0 2059    0    0]
 [   1